This notebook demonstrates how to use OptKeras, a Python package to optimize hyperparameters of Keras Deep Learning Models using Optuna.

Please see the GitHub repository of OptKeras for details:
https://github.com/Minyus/optkeras



##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
""" To use Google Drive with Colab, 
1. set use_google_drive to True, and
2. specify a directory in Google Drive (Modify as in your Google Drive)
(You will need to authorize manually.)
"""
use_google_drive = False
workdir = '/content/drive/My Drive/Colab Notebooks/OptKeras_Example_Output'


import os

try:
    if use_google_drive:
        from google.colab import drive
        drive.mount('/content/drive')
        # Create target directory & all intermediate directories if don't exists
        if not os.path.exists(workdir):
            os.makedirs(workdir)
            print('## Directory: ' , workdir ,  ' was created.') 
        os.chdir(workdir)
        print('## Current working directory: ', os.getcwd())
except:
    print('Run the code without using Google Drive.')
        
try:    
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.0162833 hours (58.62 sec)
## Check the GPU info
Fri May 10 07:31:48 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+------------------------------------------------

## Install Optuna 0.10.0

In [2]:
!pip install optuna==0.10.0

     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 81kB 17.7MB/s 
     |████████████████████████████████| 1.0MB 59.5MB/s 
     |████████████████████████████████| 102kB 40.5MB/s 
     |████████████████████████████████| 51kB 24.7MB/s 
     |████████████████████████████████| 460kB 63.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/d9/a7/05/1ecde925de6085f0c79a1dfdfed50f5a6ecd60c0d55a389890
  Stored in directory: /root/.cache/pip/wheels/de/cf/b3/0eb5c89ea6aa1b49cb41315f9ec139ada8cbffd575bf170d43
  Stored in directory: /root/.cache/pip/wheels/46/23/48/366f0d8b14d436e58ad0aef531b14af8d8beabeb2986704bd5
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna alembic Mako pyperclip


## Install OptKeras 0.0.5

In [3]:
!pip install optkeras==0.0.5
# Alternatively you can install from the GitHub repository
#!pip install git+https://github.com/Minyus/optkeras.git

## Import modules

In [4]:
import numpy as np
import pandas as pd

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

# (Optional) Disable messages from Optuna below WARN level.
optuna.logging.set_verbosity(optuna.logging.WARN) 

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.1
Optuna 0.10.0
OptKeras 0.0.5


## Set up Dataset

In [5]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## A simple Keras model

In [7]:
model = Sequential()
model.add(Conv2D(
    filters=32,
    kernel_size=3,
    strides=1,
    activation='relu',
    input_shape=input_shape ))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer=Adam(),
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train,
          validation_data=(x_test, y_test), shuffle=True,
          batch_size=512, epochs=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 7s 110us/step - loss: 0.4491 - acc: 0.8774 - val_loss: 0.2390 - val_acc: 0.9321
Epoch 2/2
60000/60000 [==============================] - 1s 19us/step - loss: 0.1955 - acc: 0.9452 - val_loss: 0.1463 - val_acc: 0.9605


## Optimization of a simple Keras model without pruning

In [8]:
study_name = dataset_name + '_Simple'

""" Step 1. Instantiate OptKeras class
You can specify arguments for Optuna's create_study method and other arguments 
for OptKeras such as enable_pruning. 
"""

ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')


""" Step 2. Define objective function for Optuna """

def objective(trial):
    
    """ Step 2.1. Define parameters to try using methods of optuna.trial such as 
    suggest_categorical. In this simple demo, try 2*2*2*2 = 16 parameter sets: 
    2 values specified in list for each of 4 parameters 
    (filters, kernel_size, strides, and activation for convolution).
    """    
    model = Sequential()
    model.add(Conv2D(
        filters=trial.suggest_categorical('filters', [32, 64]),
        kernel_size=trial.suggest_categorical('kernel_size', [3, 5]),
        strides=trial.suggest_categorical('strides', [1, 2]),
        activation=trial.suggest_categorical('activation', ['relu', 'linear']),
        input_shape=input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(),
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    """ Step 2.2. Specify callbacks(trial) and keras_verbose in fit 
    (or fit_generator) method of Keras model
    """
    model.fit(x_train, y_train, 
              validation_data=(x_test, y_test), shuffle=True,
              batch_size=512, epochs=2,
              callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose )
    
    """ Step 2.3. Return trial_best_value (or latest_value) """
    return ok.trial_best_value

""" Step 3. Run optimize. 
Set n_trials and/or timeout (in sec) for optimization by Optuna
"""
ok.optimize(objective, timeout = 3*60) # Run for 3 minutes for demo


[2019-05-10 07:32:28.281145] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)
[2019-05-10 07:32:28.583025]  (None) 
[2019-05-10 07:32:30.544185] Trial#: 0, value: 9.623000e-01| Best trial#: 0, value: 9.623000e-01, params: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-05-10 07:32:32.777806] Trial#: 1, value: 9.218000e-01| Best trial#: 0, value: 9.623000e-01, params: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-05-10 07:32:34.933685] Trial#: 2, value: 9.190000e-01| Best trial#: 0, value: 9.623000e-01, params: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-05-10 07:32:36.844214] Trial#: 3, value: 9.627000e-01| Best trial#: 3, value: 9.627000e-01, params: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-05-10 07:32:38.630483] Trial#: 4, value: 9.184000e-01| Best trial#: 3, value: 9.627000e-01, params: {'filters': 64, 'kernel_size': 5, 'strides': 2

In [9]:
""" Show Results """
print('Best trial number: ', ok.best_trial.number)
print('Best value:', ok.best_trial.value)
print('Best parameters: \n', ok.best_trial.params)

"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interrupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
print("Best parameters (retrieved directly from Optuna)", ok.study.best_trial.params)

""" Check the Optuna CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
display(pd.read_csv(ok.optuna_log_file_path))

""" Check the Keras CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
display(pd.read_csv(ok.keras_log_file_path))

Best trial number:  65
Best value: 0.9808000007629395
Best parameters: 
 {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
Best parameters (retrieved directly from Optuna) {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
Data Frame read from MNIST_Simple_Optuna.csv 



,number,state,value,datetime_start,datetime_complete,activation,filters,kernel_size,strides,_Datetime_epoch_begin,_Datetime_epoch_end,_Trial_num,acc,loss,val_acc,val_loss,system_attrs__number
0,0,TrialState.COMPLETE,0.9623,2019-05-10 07:32:28.285536,2019-05-10 07:32:30.462682,relu,64,5,2,2019-05-10 07:32:29.699403,2019-05-10 07:32:30.322589,0,0.946100,0.191616,0.9623,0.141296,0
1,1,TrialState.COMPLETE,0.9218,2019-05-10 07:32:30.463029,2019-05-10 07:32:32.696748,linear,32,3,1,2019-05-10 07:32:31.764485,2019-05-10 07:32:32.542477,1,0.918517,0.288177,0.9218,0.280232,1
2,2,TrialState.COMPLETE,0.9190,2019-05-10 07:32:32.697248,2019-05-10 07:32:34.851542,linear,32,5,1,2019-05-10 07:32:33.977338,2019-05-10 07:32:34.699587,2,0.915200,0.298365,0.9190,0.280748,2
3,3,TrialState.COMPLETE,0.9627,2019-05-10 07:32:34.852162,2019-05-10 07:32:36.761632,relu,64,5,2,2019-05-10 07:32:35.994912,2019-05-10 07:32:36.608068,3,0.946700,0.191463,0.9627,0.134839,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,TrialState.COMPLETE,0.9802,2019-05-10 07:35:17.572337,2019-05-10 07:35:20.795503,relu,64,5,1,2019-05-10 07:35:19.433082,2019-05-10 07:35:20.641565,68,0.971367,0.104053,0.9802,0.070188,68
69,69,TrialState.COMPLETE,0.9783,2019-05-10 07:35:20.804010,2019-05-10 07:35:23.901214,relu,64,5,1,2019-05-10 07:35:22.542479,2019-05-10 07:35:23.746039,69,0.972500,0.100480,0.9783,0.074368,69
70,70,TrialState.COMPLETE,0.9768,2019-05-10 07:35:23.909483,2019-05-10 07:35:27.071723,relu,64,5,1,2019-05-10 07:35:25.661064,2019-05-10 07:35:26.907323,70,0.971117,0.104899,0.9768,0.078278,70
71,71,TrialState.COMPLETE,0.9777,2019-05-10 07:35:27.081841,2019-05-10 07:35:30.375289,relu,64,5,1,2019-05-10 07:35:28.968756,2019-05-10 07:35:30.211578,71,0.971383,0.101065,0.9777,0.069256,71


Data Frame read from MNIST_Simple_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Trial_num,acc,loss,val_acc,val_loss
0,0,2019-05-10 07:32:28.812548,2019-05-10 07:32:29.642939,0,0.858783,0.532120,0.9315,0.232515
1,1,2019-05-10 07:32:29.699403,2019-05-10 07:32:30.322589,0,0.946100,0.191616,0.9623,0.141296
2,0,2019-05-10 07:32:30.764731,2019-05-10 07:32:31.711724,1,0.876700,0.438188,0.9179,0.289129
3,1,2019-05-10 07:32:31.764485,2019-05-10 07:32:32.542477,1,0.918517,0.288177,0.9218,0.280232
...,...,...,...,...,...,...,...,...
140,0,2019-05-10 07:35:24.219104,2019-05-10 07:35:25.597432,70,0.903867,0.355755,0.9658,0.131521
141,1,2019-05-10 07:35:25.661064,2019-05-10 07:35:26.907323,70,0.971117,0.104899,0.9768,0.078278
142,0,2019-05-10 07:35:27.446188,2019-05-10 07:35:28.909788,71,0.903383,0.346016,0.9655,0.127459
143,1,2019-05-10 07:35:28.968756,2019-05-10 07:35:30.211578,71,0.971383,0.101065,0.9777,0.069256


## Optimization of a Keras model using more Optuna's features such as pruning

In [10]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=optuna.samplers.TPESampler(
        consider_prior=True, prior_weight=1.0, 
        consider_magic_clip=True, consider_endpoints=False, 
        n_startup_trials=10, n_ei_candidates=24, 
        seed=None), 
    pruner=optuna.pruners.SuccessiveHalvingPruner(
        min_resource=1, reduction_factor=4, min_early_stopping_rate=0), 
    study_name=study_name,
    load_if_exists=True,
    # parameters for OptKeras
    monitor='val_acc',
    direction='maximize',
    enable_pruning=True, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1,
    )

def objective(trial): 
    epochs = 10
    
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters=int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size=batch_size,
                  epochs=epochs, validation_data=(x_test, y_test),
                  shuffle=True,
                  callbacks=ok.callbacks(trial),
                  verbose=ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks=ok.callbacks(trial),
                            verbose=ok.keras_verbose )
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, timeout=3*60) # Run for 3 minutes for demo


[2019-05-10 07:35:30.873839] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-05-10 07:35:31.526312]  (None) 
[2019-05-10 07:37:18.115806] Trial#: 0, value: 9.864000e-01| Best trial#: 0, value: 9.864000e-01, params: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 256.0, 'Optimizer': 'Adam'}
[2019-05-10 07:38:13.613516] Trial#: 3, value: 9.884000e-01| Best trial#: 3, value: 9.884000e-01, params: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 2, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters': 32.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0

## Randomized Grid Search of a simple Keras model

In [11]:
study_name = dataset_name + '_GridSearch'

""" To run randomized grid search, set random_grid_search_mode True """
ok = OptKeras(study_name=study_name, random_grid_search_mode=True)


def objective(trial):
    model = Sequential()
    model.add(Conv2D(
        filters=trial.suggest_categorical('filters', [32, 64]),
        kernel_size=trial.suggest_categorical('kernel_size', [3, 5]),
        strides=trial.suggest_categorical('strides', [1]),
        activation=trial.suggest_categorical('activation', ['relu', 'linear']),
        input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train,
              validation_data=(x_test, y_test), shuffle=True,
              batch_size=512, epochs=2,
              callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose)

    return ok.trial_best_value

""" Set the number of parameter sets as n_trials for complete grid search """
ok.random_grid_search(objective, n_trials=2*2*2)  # 2*2*2 = 8 param sets


[2019-05-10 07:40:08.445253] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)
[2019-05-10 07:40:08.525679]  (None) 
[2019-05-10 07:40:11.147222] Completed:  12% (    1 /     8)
[2019-05-10 07:40:11.222487] Trial#: 0, value: 2.863845e-01| Best trial#: 0, value: 2.863845e-01, params: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'linear'}
[2019-05-10 07:40:14.868877] Completed:  25% (    2 /     8)
[2019-05-10 07:40:14.946499] Trial#: 2, value: 2.803219e-01| Best trial#: 2, value: 2.803219e-01, params: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'linear'}
[2019-05-10 07:40:16.868072] Completed:  38% (    3 /     8)
[2019-05-10 07:40:16.943882] Trial#: 3, value: 2.810463e-01| Best trial#: 2, value: 2.803219e-01, params: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'linear'}
[2019-05-10 07:40:19.221746] Completed:  50% (    4 /     8)
[2019-05-10 07:40:19.304841] Trial#: 4, value: 1.404867e-01| Best trial#: 4, v

In [12]:
""" ## The end of code. """

' ## The end of code. '

Please feel free to post questions or feedback [here](
https://github.com/Minyus/optkeras/issues
)
